# inference patterns

short, visual examples of scoring new windows with fftboost: harmonic tracking, machine health, noise budget, and anomaly gating.

In [ ]:
# if needed:
!pip install -q git+https://github.com/pinballsurgeon/fftboost.git
!pip install -q matplotlib

import numpy as np
import matplotlib.pyplot as plt
import time

Fs = 1000
W_s = 0.4
H_s = 0.2
W = int(Fs*W_s)
H = int(Fs*H_s)
rng = np.random.default_rng(1337)

def windowize(x, W, H):
    idx = np.arange(0, len(x)-W+1, H)
    win = np.hanning(W)
    return np.stack([x[i:i+W]*win for i in idx], axis=0), idx

# ---- engineer hook: wire your fftboost model here ----
def predict_windows(iwins, fs=Fs):
    """Return model predictions for a batch of windows.
    TODO: replace with your fftboost instance's inference call, e.g.:
        return model.predict_from_windows(iwins)
    For now, provide a simple placeholder (FFT energy in target bands) so the notebook runs.
    """
    # placeholder: magnitude at fund + IH bands
    X = np.abs(np.fft.rfft(iwins, axis=1))
    f = np.fft.rfftfreq(iwins.shape[1], 1/fs)
    def band(lo, hi):
        m = (f>=lo)&(f<=hi)
        return X[:, m].sum(axis=1)
    score = 0.6*band(45,55) + 0.3*band(90,130) + 0.1*band(130,200)
    return (score - score.mean())/(score.std()+1e-9)

def ema(x, alpha=0.2):
    y = np.empty_like(x, dtype=float)
    acc = x[0]
    for i,v in enumerate(x):
        acc = alpha*v + (1-alpha)*acc
        y[i] = acc
    return y

plt.rcParams.update({"figure.figsize":(6,3)})

## 1) harmonic tracking (grid)
drifting interharmonic; show raw prediction, ema, and alarm threshold.

In [ ]:
T = 60  # seconds
t = np.arange(int(T*Fs))/Fs
x = 1.0*np.sin(2*np.pi*50*t)
x += 0.4*np.sin(2*np.pi*(100+2*np.sin(2*np.pi*0.01*t))*t)  # IH drift
x += 0.25*np.sin(2*np.pi*125*t)
x += 0.25*rng.standard_normal(len(t))

iwins, idx = windowize(x, W, H)
y = predict_windows(iwins)
y_s = ema(y, 0.2)
thr = y_s.mean() + 2*y_s.std()

tm = idx/Fs
plt.plot(tm, y, label='pred')
plt.plot(tm, y_s, label='ema')
plt.axhline(thr, ls='--', label='alarm')
plt.xlabel('time (s)'); plt.ylabel('score')
plt.title('harmonic tracking')
plt.legend(); plt.tight_layout(); plt.show()

## 2) machine health (bearing line)
fault line amplitude ramps, then drops; add waterfall and score overlay.

In [ ]:
T = 50
t = np.arange(int(T*Fs))/Fs
fault = 180
amp = np.piecewise(t, [t<20, (t>=20)&(t<40), t>=40], [0.1, 0.5, 0.15])
x = 0.9*np.sin(2*np.pi*50*t) + (amp*np.sin(2*np.pi*fault*t)) + 0.25*rng.standard_normal(len(t))
iwins, idx = windowize(x, W, H)
y = predict_windows(iwins)
tm = idx/Fs

# waterfall (coarse)
Xm = np.abs(np.fft.rfft(iwins, axis=1))
f = np.fft.rfftfreq(W, 1/Fs)
fmax = 300
fmask = f<=fmax
plt.figure(figsize=(6,3))
plt.imshow(20*np.log10(Xm[:,fmask]+1e-6).T, aspect='auto', origin='lower',
           extent=[tm[0], tm[-1], f[fmask][0], f[fmask][-1]])
plt.colorbar(label='dB')
plt.plot(tm, np.full_like(tm, fault), 'w--', linewidth=0.8)
plt.xlabel('time (s)'); plt.ylabel('Hz'); plt.title('waterfall + score')
ax = plt.gca().twinx()
ax.plot(tm, (y-y.min())/(np.ptp(y)+1e-9), 'r-', alpha=0.7)
ax.set_ylabel('norm. score')
plt.tight_layout(); plt.show()

## 3) noise budget (band attribution)
perturb HF noise and show bandwise contributions to the score.

In [ ]:
def synth_noise_case(hf_scale):
    t = np.arange(W)/Fs
    w = np.hanning(W)
    s = 1.0*np.sin(2*np.pi*50*t) + 0.4*np.sin(2*np.pi*100*t) + 0.2*np.sin(2*np.pi*125*t)
    s += hf_scale*rng.standard_normal(W)
    return s*w

cases = [0.1, 0.3, 0.6, 1.0]
bands = [(45,55),(90,130),(130,200),(200,400)]
labels = ['fund','IH1','IH2','HF']
contrib = []

for c in cases:
    iw = synth_noise_case(c)[None,:]
    X = np.abs(np.fft.rfft(iw, axis=1)); f = np.fft.rfftfreq(W,1/Fs)
    parts = []
    for lo,hi in bands:
        m = (f>=lo)&(f<=hi)
        parts.append(float(X[:,m].sum()))
    contrib.append(parts)

contrib = np.array(contrib)
x = np.arange(len(labels))
width = 0.18
plt.figure(figsize=(6,3))
for i,c in enumerate(cases):
    plt.bar(x + i*width, contrib[i], width, label=f'hf={c}')
plt.xticks(x + width*1.5, labels)
plt.ylabel('band energy (a.u.)'); plt.title('band attribution vs HF noise')
plt.legend(ncol=len(cases), fontsize=8)
plt.tight_layout(); plt.show()

## 4) anomaly gate with uncertainty (bootstrap)
estimate CI via resampling active bands; flag outliers by z-score.

In [ ]:
def fake_active_bins():
    # placeholder: emulate that the model uses bins near {50,100,125} Hz
    f = np.fft.rfftfreq(W, 1/Fs)
    bins = []
    for target in [50,100,125]:
        bins.append(int(np.argmin(np.abs(f - target))))
    return np.array(bins)
active_bins = fake_active_bins()

# build a stream with occasional anomalies
T = 80
t = np.arange(int(T*Fs))/Fs
x = 1.0*np.sin(2*np.pi*50*t) + 0.25*np.sin(2*np.pi*125*t) + 0.25*rng.standard_normal(len(t))
an_idx = (t>30)&(t<32)
x[an_idx] += 0.8*np.sin(2*np.pi*100*t[an_idx])  # injected anomaly

iwins, idx = windowize(x, W, H)
scores = predict_windows(iwins)

def bootstrap_ci(iwins, bins, B=64):
    # crude: resample active bins with replacement and recompute summed magnitude
    X = np.abs(np.fft.rfft(iwins, axis=1))
    vals = []
    for _ in range(B):
        sel = rng.choice(bins, size=len(bins), replace=True)
        vals.append(X[:, sel].sum(axis=1))
    vals = np.stack(vals, axis=1)
    lo = np.percentile(vals, 2.5, axis=1)
    hi = np.percentile(vals, 97.5, axis=1)
    return lo, hi

lo, hi = bootstrap_ci(iwins, active_bins, B=64)
z = (scores - scores.mean())/(scores.std()+1e-9)
tm = idx/Fs

fig, ax = plt.subplots(1,1, figsize=(6,3))
ax.plot(tm, scores, label='score')
ax.fill_between(tm, lo, hi, color='C1', alpha=0.2, label='bootstrap CI (proxy)')
ax.plot(tm[z>2.5], scores[z>2.5], 'rx', label='anomaly (z>2.5)')
ax.set_xlabel('time (s)'); ax.set_ylabel('score'); ax.set_title('anomaly gate with CI')
ax.legend(); plt.tight_layout(); plt.show()

## latency snapshot
approximate per-window inference time for your hook function.

In [ ]:
batch = rng.standard_normal((256, W)) * np.hanning(W)
t0 = time.perf_counter()
y = predict_windows(batch)
t1 = time.perf_counter()
print(f"latency ~ {(t1-t0)*1e3/len(batch):.3f} ms / window (hook)")